In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
import hashlib
from PIL import Image
from io import BytesIO

In [32]:
def setup_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")  # newer headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--log-level=3")
    options.add_argument("--no-sandbox")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [33]:
def download_image(url, folder):
    try:
        image_content = requests.get(url, timeout=10).content
        image = Image.open(BytesIO(image_content)).convert("RGB")
        file_path = os.path.join(folder, hashlib.sha1(image_content).hexdigest() + ".jpg")
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"Downloaded: {file_path}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")


In [34]:
def scrape_images(query, limit=10, target_dir='downloads'):
    folder = os.path.join(target_dir, '_'.join(query.lower().split()))
    os.makedirs(folder, exist_ok=True)

    driver = setup_driver(headless=False)
    driver.get(f"https://www.google.com/search?q={query}&tbm=isch")

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img.Q4LuWd"))
        )
    except:
        print("No images loaded.")
        driver.quit()
        return

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    imgs = driver.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
    print(f"🖼️ Found {len(imgs)} image tags")

    count = 0
    for img in imgs:
        if count >= limit:
            break
        try:
            src = img.get_attribute("src") or img.get_attribute("data-src")
            if src and src.startswith("http"):
                download_image(src, folder)
                count += 1
        except Exception as e:
            print("Skipping image:", e)

    driver.quit()
    print("✅ Scraping finished.")

In [36]:
scrape_images("neymar", limit=30)

No images loaded.
